In [1]:
import os
import numpy as np

In [2]:
# convert the simple html to dict/json format
def convert_to_json(text,index):
    state = 0
    key = ""
    key2 = ""
    data_json = {}
    subresult = ""
    old = index
    while index<len(text):
        l = text[index]
        index+=1
        if state==0:
            if l=='<':
                state=1
        elif state==1:
            if l=="/":
                s = ""
                if not bool(data_json):
                    for i in range(old,index-2):
                        s+=text[i]
                    return s.strip(),index-2
                return data_json,index-2
            elif l=='>':
                subresult,index=convert_to_json(text,index)
                state=2
            else:
                key+=l
        elif state==2:
            if l=='<':
                state=3
        else:
            if l=='>':
                if key in data_json:
                    c = 1
                    while key+"_"+str(c)  in data_json:
                        c+=1
                    key = key+"_"+str(c)
                data_json[key] = subresult
                key = ""
                key2= ""
                subresult = ""
                state = 0
            else:
                key2+=l
    return data_json,index

In [3]:
def extractInfo(f):
    text = open(f).read()
    data_json,_ = convert_to_json(np.array(list(text.strip())),0)
    return data_json

In [4]:
# Data for Task1
path = "lot-parser/data/2015-03-18/"
filepaths = []
for paths,folders,files in os.walk(path):
    if len(files)>0:
        for file in files:
            if file.endswith('.html'):
                filepaths.append(paths+'/'+file)

In [5]:
# artist names
def script1(filepaths):
    result = []
    for file in filepaths:
        data = extractInfo(file)
        result.append(data['html']['body']['h2'])
    return result

In [6]:
script1(filepaths)

['Rembrandt Harmensz. van Rijn',
 'Joan Miró',
 'Marc Chagall',
 'Louis Marcoussis (1883-1941)',
 'Pablo Picasso']

In [7]:
def prepData(file,artist_index,artist_count,result):
    data = extractInfo(file)['html']
    temp = {}
    temp['artist'] = data['body']['h2']
    temp['works'] = []
    index = artist_count
    if temp['artist'] not in artist_index:
        result.append(temp)
        artist_index[temp['artist']] = artist_count
        artist_count+=1
    else:
        index = artist_index[temp['artist']]
    return artist_index,artist_count,index,data

In [8]:
# artist names
def script2(filepaths):
    result = []
    artist_index = {}
    artist_count = 0
    for file in filepaths:
        artist_index,artist_count,index,data = prepData(file,artist_index,artist_count,result)
        result[index]['works'].append(data['title'])
    return result

In [9]:
script2(filepaths)

[{'artist': 'Rembrandt Harmensz. van Rijn',
  'works': ['Rembrandt Harmensz. van Rijn: Christ at Emmaus: The smaller Plate']},
 {'artist': 'Joan Miró',
  'works': ['Joan Miró: Femme et Chien devant la Lune']},
 {'artist': 'Marc Chagall', 'works': ['Marc Chagall: Self-Portrait']},
 {'artist': 'Louis Marcoussis (1883-1941)',
  'works': ['Louis Marcoussis: La Table']},
 {'artist': 'Pablo Picasso',
  'works': ['Pablo Picasso: Quatre Femmes nues et Tête sculptée, from: La Suite Vollard']}]

In [10]:
# artist names
def script3(filepaths):
    result = []
    artist_index = {}
    artist_count = 0
    for file in filepaths:
        artist_index,artist_count,index,data = prepData(file,artist_index,artist_count,result)
        result[index]['works'].append({'title': data['title'], 'price': data['body']['div_1']})
    return result

In [11]:
script3(filepaths)

[{'artist': 'Rembrandt Harmensz. van Rijn',
  'works': [{'price': 'GBP 6,875',
    'title': 'Rembrandt Harmensz. van Rijn: Christ at Emmaus: The smaller Plate'}]},
 {'artist': 'Joan Miró',
  'works': [{'price': 'GBP 11,250',
    'title': 'Joan Miró: Femme et Chien devant la Lune'}]},
 {'artist': 'Marc Chagall',
  'works': [{'price': 'GBP 6,000', 'title': 'Marc Chagall: Self-Portrait'}]},
 {'artist': 'Louis Marcoussis (1883-1941)',
  'works': [{'price': 'GBP 9,200', 'title': 'Louis Marcoussis: La Table'}]},
 {'artist': 'Pablo Picasso',
  'works': [{'price': 'USD 25,000',
    'title': 'Pablo Picasso: Quatre Femmes nues et Tête sculptée, from: La Suite Vollard'}]}]

In [12]:
# artist names
def script4(filepaths):
    result = []
    artist_index = {}
    artist_count = 0
    for file in filepaths:
        artist_index,artist_count,index,data = prepData(file,artist_index,artist_count,result)
        currency, amount = data['body']['div_1'].split()
        result[index]['works'].append({'title': data['title'], 'currency': currency, 'amount': amount })
    return result

In [13]:
script4(filepaths)

[{'artist': 'Rembrandt Harmensz. van Rijn',
  'works': [{'amount': '6,875',
    'currency': 'GBP',
    'title': 'Rembrandt Harmensz. van Rijn: Christ at Emmaus: The smaller Plate'}]},
 {'artist': 'Joan Miró',
  'works': [{'amount': '11,250',
    'currency': 'GBP',
    'title': 'Joan Miró: Femme et Chien devant la Lune'}]},
 {'artist': 'Marc Chagall',
  'works': [{'amount': '6,000',
    'currency': 'GBP',
    'title': 'Marc Chagall: Self-Portrait'}]},
 {'artist': 'Louis Marcoussis (1883-1941)',
  'works': [{'amount': '9,200',
    'currency': 'GBP',
    'title': 'Louis Marcoussis: La Table'}]},
 {'artist': 'Pablo Picasso',
  'works': [{'amount': '25,000',
    'currency': 'USD',
    'title': 'Pablo Picasso: Quatre Femmes nues et Tête sculptée, from: La Suite Vollard'}]}]